In [4]:
import requests
from bs4 import BeautifulSoup
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
def scrape_offers_requests(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    offers = []

    # chaque offre = une ligne <tr id="O....">
    for row in soup.select("tr[id^='O']"):

        link = row.select_one("a.lien-details-offre")
        if not link:
            continue

        offer_title = link.get_text(strip=True)
        offer_url = "https://www.emploi-territorial.fr" + link["href"]

        offers.append({
            "title": offer_title,
            "url": offer_url
        })

    return offers

In [5]:
def scrape_offer_detail(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Conteneur principal du détail de l'offre
    main = soup.select_one("div#contenuOffre, div.contenu-offre, main")

    if not main:
        return ""

    # Supprimer éléments inutiles
    for tag in main(["script", "style", "button", "nav"]):
        tag.decompose()

    text = main.get_text(separator="\n", strip=True)
    return text


In [6]:
URL = "https://www.emploi-territorial.fr/emploi-mobilite/?search-fam-metier=A7&page=20000"

offers = scrape_offers_requests(URL)

corpus = []

for offer in offers:
    print(f"Scraping : {offer['title']}")
    full_text = scrape_offer_detail(offer["url"])

    corpus.append({
        "title": offer["title"],
        "url": offer["url"],
        "text": full_text
    })

    time.sleep(1)  # pour éviter les surchages même si on récupère pas 10K emplois d'un coup



Scraping : Technicien support et déploiement (h/f)
Scraping : Chef de projet informatique
Scraping : Ingénieur Bases de données
Scraping : Adjoint responsable pôle maintenance du service informatique
Scraping : Technicien informatique  (h/f)
Scraping : Assistant à Maîtrise d'Ouvrage SI (AMOA) (h/f)
Scraping : Technicien Système Information Géographique h/f
Scraping : OPERATEUR EN SIMULATION VIRTUELLE
Scraping : Développeur O365
Scraping : Technicien SIG (h/f)
Scraping : Un ou une responsable d'offre de service Mobilité
Scraping : Responsable service ressources et direction des systèmes d’information
Scraping : DES ASSISTANTS EN MAINTENANCE INFORMATIQUE (H/F) - ZONE SUD
Scraping : ADMINISTRATEUR SYSTEMES ET RESEAUX
Scraping : Responsable de service informatique F/H
Scraping : 1 AGENT ADMINISTRATIF, (TIT ou CONT) ORGANISATION DES EVENEMENTS ET COMMUNICATION RESEAUX
Scraping : Référent.e de proximité SI MDPH - Offre 3849
Scraping : Chargé de Projet Système Informations Géographiques  (F/H

In [7]:
import csv

def save_corpus_to_csv(corpus, filename="corpus_offres_data.csv"):
    with open(filename, mode="w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["title", "url", "text"],
            delimiter=";"
        )
        writer.writeheader()
        for row in corpus:
            writer.writerow(row)
save_corpus_to_csv(corpus)

# petite sauvegarde en csv rapide pour pas le perdre (optionnelle pour la suite)
